# Weapon Stats

## Setting Up

In [1]:
import pandas as pd

%run weapon_full_data.ipynb

TypeError: '<' not supported between instances of 'NoneType' and 'int'

TypeError: '<' not supported between instances of 'NoneType' and 'int'

## Damage

### Base Damage

How much damage a weapon can make in a single turn.

In [ ]:
average_damage_mean = weapon_data[["weapon", "turn_damage_expected"]]
average_damage_mean = average_damage_mean.groupby("weapon").mean()

In [ ]:
average_damage_max = weapon_data[["weapon", "turn_damage_max"]]
average_damage_max = average_damage_max.groupby("weapon").mean()

In [ ]:
average_damage = average_damage_max.merge(average_damage_mean, on="weapon")
average_damage = average_damage.sort_values(by=["turn_damage_expected"])

In [ ]:
ax = average_damage.plot.bar(title="Base damage", legend=False)
ax.legend(["Max", "Average"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Damage")

### Penetrating Damage

In [ ]:
average_damage_mean = weapon_data_aliens[["weapon", "turn_penetrating_damage_expected"]]
average_damage_mean = average_damage_mean.groupby("weapon").mean()

In [ ]:
average_damage_max = weapon_data_aliens[["weapon", "turn_penetrating_damage_max"]]
average_damage_max = average_damage_max.groupby("weapon").mean()

In [ ]:
average_damage = average_damage_max.merge(average_damage_mean, on="weapon")
average_damage = average_damage.sort_values(by=["turn_penetrating_damage_expected"])

In [ ]:
ax = average_damage.plot.bar(title="Penetrating damage", legend=False)
ax.legend(["Max", "Average"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Damage")

### Base vs Penetrating Damage

In [ ]:
average_damage_mean = weapon_data[["weapon", "turn_damage_expected"]]
average_damage_mean = average_damage_mean.groupby("weapon").mean()

In [ ]:
average_penetrating_damage_mean = weapon_data_aliens[["weapon", "turn_penetrating_damage_expected"]]
average_penetrating_damage_mean = average_penetrating_damage_mean.groupby("weapon").mean()

In [ ]:
average_damage = average_damage_mean.merge(average_penetrating_damage_mean, on="weapon")
average_damage = average_damage.sort_values(by=["turn_penetrating_damage_expected"])

In [ ]:
ax = average_damage.plot.bar(title="Expected damage", legend=False)
ax.legend(["Base", "Real"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Damage")

In [ ]:
average_damage_mean = weapon_data[["weapon", "turn_damage_max"]]
average_damage_mean = average_damage_mean.groupby("weapon").mean()

In [ ]:
average_penetrating_damage_mean = weapon_data_aliens[["weapon", "turn_penetrating_damage_max"]]
average_penetrating_damage_mean = average_penetrating_damage_mean.groupby("weapon").mean()

In [ ]:
average_damage = average_damage_mean.merge(average_penetrating_damage_mean, on="weapon")
average_damage = average_damage.sort_values(by=["turn_penetrating_damage_max"])

In [ ]:
ax = average_damage.plot.bar(title="Max damage", legend=False)
ax.legend(["Base", "Real"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Damage")

## Ammo

In [ ]:
weapon_ammo = weapon_data[["weapon", "capacity"]]
weapon_ammo = weapon_ammo.rename(columns={"capacity": "ammo"})
weapon_ammo = weapon_ammo.groupby("weapon").max()
weapon_ammo = weapon_ammo.sort_values(by=["ammo"])

# Remove weapons with unlimited ammo
weapon_ammo = weapon_ammo[weapon_ammo["ammo"] < 255]

weapon_ammo

In [ ]:
ax = weapon_ammo.plot.bar(title="Ammo", legend=False)
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Shots")

## Damage Per Clip

In [ ]:
damage_per_clip = weapon_data[["weapon", "clip_damage"]]
damage_per_clip = damage_per_clip.groupby("weapon").max()
damage_per_clip = damage_per_clip.sort_values(by=["clip_damage"])
damage_per_clip = damage_per_clip.dropna()

In [ ]:
penetrating_damage_per_clip = weapon_data_aliens[["weapon", "clip_penetrating_damage"]]
penetrating_damage_per_clip = penetrating_damage_per_clip.groupby("weapon").max()
penetrating_damage_per_clip = penetrating_damage_per_clip.sort_values(by=["clip_penetrating_damage"])
penetrating_damage_per_clip = penetrating_damage_per_clip.dropna()

In [ ]:
# Remove weapons with unlimited ammo
damage_per_clip = damage_per_clip.merge(penetrating_damage_per_clip, on="weapon")
damage_per_clip = damage_per_clip[damage_per_clip["clip_damage"] < 10000]

damage_per_clip

In [ ]:
ax = damage_per_clip.plot.bar(title="Damage per clip", legend=False)
ax.legend(["Base", "Real"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Damage")

## Time Units

How much does it cost each use of a weapon? Cost is measured in Time Units.

In [ ]:
time_units_max = weapon_data[["weapon", "time_units"]]
time_units_max = time_units_max.groupby("weapon").max()

In [ ]:
time_units_average = weapon_data[["weapon", "time_units"]]
time_units_average = time_units_average.groupby("weapon").mean()

In [ ]:
time_units = time_units_max.merge(time_units_average, on="weapon")
time_units = time_units.rename(columns={"time_units_x": "time_units_max", "time_units_y": "time_units_avg"})
time_units = time_units.sort_values(by=["time_units_avg"])
time_units

In [ ]:
ax = time_units.plot.bar(title="Time Units Cost")
ax.legend(["Max", "Average"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("TUs")

## Shots Per Turn

Supposing all the attacks are done on auto, or the highest rate of fire setting.

In [ ]:
shots_per_turn_max = weapon_data[["weapon", "turn_max_shots"]]
shots_per_turn_max = shots_per_turn_max.groupby("weapon").max()

In [ ]:
shots_per_turn_average = weapon_data[["weapon", "turn_max_shots"]]
shots_per_turn_average = shots_per_turn_average.groupby("weapon").mean()
shots_per_turn_average["turn_max_shots"] = shots_per_turn_average.apply(lambda x: int(x["turn_max_shots"]), axis=1)

In [ ]:
shots_per_turn = shots_per_turn_max.merge(shots_per_turn_average, on="weapon")
shots_per_turn = shots_per_turn.rename(columns={"turn_max_shots_x": "turn_shots_max", "turn_max_shots_y": "turn_shots_avg"})
shots_per_turn = shots_per_turn.sort_values(by=["turn_shots_avg"])

In [ ]:
ax = shots_per_turn.plot.bar(title="Shots per turn")
ax.legend(["Max", "Average"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Shots")

## Turns of Sustained Fire

Weapons with unlimited ammo are not taken into account.

In [ ]:
turns_sustained_fire_max = weapon_data[["weapon", "turns_to_reload"]]
turns_sustained_fire_max = turns_sustained_fire_max.groupby("weapon").max()

# Remove weapons with unlimited ammo
turns_sustained_fire_max = turns_sustained_fire_max[turns_sustained_fire_max["turns_to_reload"] < 70]

In [ ]:
turns_sustained_fire_average = weapon_data[["weapon", "turns_to_reload"]]
turns_sustained_fire_average = turns_sustained_fire_average.groupby("weapon").mean()

# Remove weapons with unlimited ammo
turns_sustained_fire_average = turns_sustained_fire_average[turns_sustained_fire_average["turns_to_reload"] < 70]

In [ ]:
turns_sustained_fire = turns_sustained_fire_max.merge(turns_sustained_fire_average, on="weapon")
turns_sustained_fire = turns_sustained_fire.rename(columns={"turns_to_reload_x": "turns_sustained_fire_max", "turns_to_reload_y": "turns_sustained_fire_avg"})
turns_sustained_fire = turns_sustained_fire.sort_values(by=["turns_sustained_fire_avg"])

In [ ]:
ax = turns_sustained_fire.plot.bar(title="Turns of sustained fire")
ax.legend(["Max", "Average"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Turns")

## Damage Per Time Unit

### Base Damage Per Time Unit

In [ ]:
damage_per_tu_max = weapon_data[["weapon", "damage_per_time_unit"]]
damage_per_tu_max = damage_per_tu_max.groupby("weapon").max()

In [ ]:
damage_per_tu_average = weapon_data[["weapon", "damage_per_time_unit"]]
damage_per_tu_average = damage_per_tu_average.groupby("weapon").mean()

In [ ]:
damage_per_tu = damage_per_tu_max.merge(damage_per_tu_average, on="weapon")
damage_per_tu = damage_per_tu.rename(columns={"damage_per_time_unit_x": "damage_per_time_unit_max", "damage_per_time_unit_y": "damage_per_time_unit_avg"})
damage_per_tu = damage_per_tu.sort_values(by=["damage_per_time_unit_avg"])

In [ ]:
ax = damage_per_tu.plot.bar(title="Damage per TU")
ax.legend(["Max", "Average"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Ratio")

### Penetrating Damage Per Time Unit

In [ ]:
damage_per_tu_max = weapon_data_aliens[["weapon", "penetrating_damage_per_time_unit"]]
damage_per_tu_max = damage_per_tu_max.groupby("weapon").max()

In [ ]:
damage_per_tu_average = weapon_data_aliens[["weapon", "penetrating_damage_per_time_unit"]]
damage_per_tu_average = damage_per_tu_average.groupby("weapon").mean()

In [ ]:
damage_per_tu = damage_per_tu_max.merge(damage_per_tu_average, on="weapon")
damage_per_tu = damage_per_tu.rename(columns={"penetrating_damage_per_time_unit_x": "damage_per_time_unit_max", "penetrating_damage_per_time_unit_y": "damage_per_time_unit_avg"})
damage_per_tu = damage_per_tu.sort_values(by=["damage_per_time_unit_avg"])

In [ ]:
ax = damage_per_tu.plot.bar(title="Damage per TU")
ax.legend(["Max", "Average"])
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Ratio")

## Hits to Kill

In [ ]:
hits_to_kill_average = weapon_data_aliens[["weapon", "hits_to_kill"]]
hits_to_kill_average = hits_to_kill_average[hits_to_kill_average["hits_to_kill"] < 200]
hits_to_kill_average = hits_to_kill_average.groupby(["weapon"]).mean()
hits_to_kill_average["hits_to_kill"] = hits_to_kill_average.apply(lambda x: math.ceil(x["hits_to_kill"]), axis=1)
hits_to_kill_average = hits_to_kill_average.sort_values(by=["hits_to_kill"])

In [ ]:
ax = hits_to_kill_average.plot.bar(title="Hits to kill on average alien", legend=False)
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Hits")

### Expected Kills in a Turn

In [ ]:
expected_kills = weapon_data_aliens[["weapon", "turn_expected_kills"]]
expected_kills = expected_kills.groupby(["weapon"]).mean()
expected_kills["turn_expected_kills"] = expected_kills.apply(lambda x: math.ceil(x["turn_expected_kills"]), axis=1)
expected_kills = expected_kills.sort_values(by=["turn_expected_kills"])

In [ ]:
ax = expected_kills.plot.bar(title="Expected kills on sustained fire", legend=False)
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Kills")

## Overkill

In [ ]:
overkill = weapon_data_aliens[["weapon", "overkill"]]
overkill = overkill.groupby(["weapon"]).mean()
overkill = overkill.sort_values(by=["overkill"])

In [ ]:
ax = overkill.plot.bar(title="Overkill per death", legend=False)
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("Overkill")

### Time Units Wasted on Overkill

In [ ]:
overkill = weapon_data_aliens[["weapon", "overkill_per_time_unit"]]
overkill = overkill.groupby(["weapon"]).mean()
overkill = overkill.sort_values(by=["overkill_per_time_unit"])

In [ ]:
ax = overkill.plot.bar(title="TUs wasted per kill", legend=False)
ax.xaxis.get_label().set_visible(False)
ax.set_ylabel("TUs")